<a href="https://colab.research.google.com/github/ElenaBetelgeuse/Web-scraping/blob/main/%D0%9E%D1%81%D0%BD%D0%BE%D0%B2%D1%8B_%D0%B2%D0%B5%D0%B1_%D1%81%D0%BA%D1%80%D0%B0%D0%BF%D0%B8%D0%BD%D0%B3%D0%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ОПИСАНИЕ ЗАДАЧИ

# Вам необходимо написать функцию, которая будет основана на поиске по сайту habr.com.
# Функция в качестве параметра должна принимать список запросов для поиска (например, ['python', 'анализ данных']) и на основе материалов,
# попавших в результаты поиска по каждому запросу, возвращать датафрейм вида:
# <дата> - <заголовок> - <ссылка на материал>
# В рамках задания предполагается работа только с одной (первой) страницей результатов поисковой выдачи для каждого запроса.
# Материалы в датафрейме не должны дублироваться, если они попадали в результаты поиска для нескольких запросов из списка.
# Функция из обязательной части задания должна быть расширена следующим образом:
# кроме списка ключевых слов для поиска необходимо объявить параметр с количеством страниц поисковой выдачи.
# Т.е. при передаче в функцию аргумента 4 необходимо получить материалы с первых 4 страниц результатов;
# в датафрейме должны быть столбцы с полным текстом найденных материалов и количеством лайков:
# <дата> - <заголовок> - <ссылка на материал> - <текст материала> - <количество лайков>

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import re

In [ ]:
def get_habr(query, pages):
    habr_search = pd.DataFrame()
    for i in range(1, pages+1):
        for q in query:
            word = re.sub('\s+', '%20', q)
            if i == 1:
               pages_num = ''
            else: pages_num = 'page' + str(i) + '/'
            URL = 'https://habr.com/ru/search/' + pages_num + '?q=' + word + '&target_type=posts&order=relevance'

            req = requests.get(URL)
            time.sleep(0.3)
            soup = BeautifulSoup(req.text)
            articles = soup.find_all('article', class_='tm-articles-list__item')
            for article in articles:
                if article.find('span', class_='tm-article-snippet__datetime-published'):
                    date = article.find('span', class_='tm-article-snippet__datetime-published').text
                else:
                    date = article.find('a', class_='tm-megapost-snippet__link tm-megapost-snippet__date').text
                    title = article.find('a', class_='tm-megapost-snippet__link tm-megapost-snippet__card').text.strip()
                    link = 'https://habr.com'+ article.find('a', class_='tm-megapost-snippet__link tm-megapost-snippet__card').get('href')
                    req_text = requests.get(link).text
                    soup_text = BeautifulSoup(req_text)
                    if soup_text.find('div', class_='tm-article-body tm-megapost-presenter__megapost-html'):
                       text = soup_text.find('div', class_='tm-article-body tm-megapost-presenter__megapost-html').text.strip()
                    else: text = 'СТАТЬЯ УДАЛЕНА ИЛИ ПЕРЕМЕЩЕНА'
                    if article.find('span', class_='tm-votes-meter__value tm-votes-meter__value tm-votes-meter__value_positive tm-votes-meter__value_appearance-article tm-votes-meter__value_rating'):
                       likes = article.find('span', class_='tm-votes-meter__value tm-votes-meter__value tm-votes-meter__value_positive tm-votes-meter__value_appearance-article tm-votes-meter__value_rating').text
                    elif article.find('span', class_='tm-votes-meter__value tm-votes-meter__value tm-votes-meter__value_appearance-article tm-votes-meter__value_rating'):
                        likes = article.find('span', class_='tm-votes-meter__value tm-votes-meter__value tm-votes-meter__value_appearance-article tm-votes-meter__value_rating').text
                    else:   likes = article.find('span', class_='tm-votes-meter__value tm-votes-meter__value tm-votes-meter__value_negative tm-votes-meter__value_appearance-article tm-votes-meter__value_rating').text
                    row = {'ДАТА': date, 'ЗАГОЛОВОК': title, 'ССЫЛКА НА МАТЕРИАЛ': link, 'ТЕКСТ МАТЕРИАЛА' : text, 'КОЛИЧЕСТВО ЛАЙКОВ' : likes }
                    habr_search = pd.concat([habr_search, pd.DataFrame([row])])
                    continue
                title = article.find('h2', class_='tm-article-snippet__title').text.strip()
                link = 'https://habr.com'+ article.find('a', class_='tm-article-snippet__title-link').get('href')
                req_text = requests.get(link).text
                soup_text = BeautifulSoup(req_text)
                if soup_text.find('article', class_='tm-article-presenter__content tm-article-presenter__content_narrow'):
                   text = soup_text.find('article', class_='tm-article-presenter__content tm-article-presenter__content_narrow').text.strip()
                else: text = 'СТАТЬЯ УДАЛЕНА ИЛИ ПЕРЕМЕЩЕНА'
                if article.find('span', class_='tm-votes-meter__value tm-votes-meter__value tm-votes-meter__value_positive tm-votes-meter__value_appearance-article tm-votes-meter__value_rating'):
                   likes = article.find('span', class_='tm-votes-meter__value tm-votes-meter__value tm-votes-meter__value_positive tm-votes-meter__value_appearance-article tm-votes-meter__value_rating').text
                elif article.find('span', class_='tm-votes-meter__value tm-votes-meter__value tm-votes-meter__value_appearance-article tm-votes-meter__value_rating'):
                    likes = article.find('span', class_='tm-votes-meter__value tm-votes-meter__value tm-votes-meter__value_appearance-article tm-votes-meter__value_rating').text
                else:   likes = article.find('span', class_='tm-votes-meter__value tm-votes-meter__value tm-votes-meter__value_negative tm-votes-meter__value_appearance-article tm-votes-meter__value_rating').text

                row = {'ДАТА': date, 'ЗАГОЛОВОК': title, 'ССЫЛКА НА МАТЕРИАЛ': link, 'ТЕКСТ МАТЕРИАЛА' : text, 'КОЛИЧЕСТВО ЛАЙКОВ' : likes }
                habr_search = pd.concat([habr_search, pd.DataFrame([row])])
    habr_search_dropped = habr_search.drop_duplicates (subset=['ССЫЛКА НА МАТЕРИАЛ'])
    return habr_search_dropped.reset_index(drop=True)


res = get_habr(['python', 'анализ данных'], 2)
print(len(res))
res

79


,ДАТА,ЗАГОЛОВОК,ССЫЛКА НА МАТЕРИАЛ,ТЕКСТ МАТЕРИАЛА,КОЛИЧЕСТВО ЛАЙКОВ
0,20 января 2022 в 18:37,Курс «Python для инженеров». Старт 3 потока 31...,https://habr.com/ru/company/southbridge/news/t...,Hedgehog_art\n 20 января 2022 в 18:37...,+10
1,13 декабря 2021 в 09:00,Жаждущим автоматизации: открытый урок «ChatOps...,https://habr.com/ru/company/southbridge/news/t...,geniyoctober\n 13 декабря 2021 в 09:0...,+9
2,21 апреля 2020 в 18:35,"Вышел Python 2.7.18, последний релиз ветки Pyt...",https://habr.com/ru/news/t/498364/,denis-19\n 21 апреля 2020 в 18:35 Вы...,+19
3,6 июля 2021 в 13:29,Python Community Meetup 8/07: видео и материал...,https://habr.com/ru/company/raiffeisenbank/new...,Raiffeisenbank\n 6 июля 2021 в 13:29 ...,+3
4,13 января 2022 в 18:35,Открытый урок «Пишем Custom Prometheus Exporte...,https://habr.com/ru/company/southbridge/news/t...,Hedgehog_art\n 13 января 2022 в 18:35...,+10
...,...,...,...,...,...
74,14 декабря 2016 в 13:25,Как хорошо подать результаты качественного исс...,https://habr.com/ru/post/317544/,TamaraK\n 14 декабря 2016 в 13:25 Ка...,+12
75,7 октября 2013 в 14:12,Введение в анализ сложности алгоритмов (часть 1),https://habr.com/ru/post/196560/,AveNat\n 7 октября 2013 в 14:12 Введ...,+90
76,20 октября 2015 в 17:30,Совмещенный АВС и XYZ анализ в Ритейле,https://habr.com/ru/company/datawiz/blog/269167/,jinxu\n 20 октября 2015 в 17:30 Совм...,+14
77,6 сентября 2019 в 13:52,Как превратить трафик в продажи с помощью данн...,https://habr.com/ru/post/466475/,NadyaRookee\n 6 сентября 2019 в 13:52...,+9
